In [1]:
import configparser

config = configparser.ConfigParser(interpolation=None)
config.read('config.ini')

openai_api_key   = config['openai']['api_key']

assistant_id = config['openai-assistant']['assistant_id']

In [22]:
from openai import OpenAI


client = OpenAI(api_key=openai_api_key)

#provide the assisatne id bellow
assistant = client.beta.assistants.retrieve(assistant_id)


In [3]:
def ask_assistant(question):
    thread = client.beta.threads.create()
    
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=question
    )
    
    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )
    
    while run.status != 'completed':
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    return messages.data[0].content[0].text.value

In [36]:
answer = ask_assistant("name xuebin, body: when is the final exam ")
print(answer)

Hello Xuebin,

The final exam for IA340 Intro to Database class is scheduled for December 12th, 2023. Please make sure to review all the materials and practice the exercises provided throughout the course to prepare effectively.

If you have any more questions or need further assistance, feel free to reach out.

Best regards,

[Your Name]
IA340 Intro to Database TA


In [35]:
import win32com.client
import re

def extract_name(email):
    pattern = r"^([^@]+)@"
    match = re.search(pattern, email)
    return match.group(1) if match else email

def process_emails(sender_filter):
    outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    inbox = outlook.GetDefaultFolder(6)
    messages = inbox.Items
    messages.Sort("[ReceivedTime]", True)

    for message in messages:
        if sender_filter.lower() in message.SenderEmailAddress.lower():

            sender_name = message.SenderName
            body_text = message.Body

            print(f"Sender: {sender_name}")
            print(f"Body: {body_text[:100]}...")  # Print first 100 characters

            reply = message.Reply()
            reply_body = ask_assistant(f"sender: {sender_name}, body:{body_text}")
            reply.Body = f"{reply_body}\n\n--- Original Message ---\n{message.Body}"
          
            reply.Save()

    


In [34]:
sender_filter = "jmu.edu"  # Replace with the desired sender domain or email
process_emails(sender_filter)

Sender: auto_notification@jmu.edu
Body: Fall 2024 Regular and 7W2 final grade rosters are now available for grade entry on MyMadison. Please...
Sender: copier@jmu.edu
Body: ...
Sender: copier@jmu.edu
Body: ...
Sender: copier@jmu.edu
Body: ...
Sender: copier@jmu.edu
Body: ...
